In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('test.csv')

df.shape
df.head()

(6505381, 13)

,user_id,account_id,owner_id,order_no,ticket_price,ticket_num,entry_station_code,exit_station_code,order_status,reg_date,entry_date,entry_station,exit_station
0,NaN,NaN,ort1DswjyPc6pUEzGuJcgNepO9E0,1201702070627252238,800,3,317,3002.0,5,2017-02-07 06:27:25,2017-02-07 06:28:15,岗顶,机场南
1,NaN,NaN,ort1Ds1jeqEkh9eHBc5FC0Pec_qI,1201702081959049604,200,1,107,0.0,5,2017-02-08 19:59:04,2017-02-08 20:01:15,陈家祠,NaN
2,NaN,NaN,ort1Ds9TrCcK3YmzkCq3XoNqkIVA,1201702171545377297,400,1,310,312.0,5,2017-02-17 15:45:36,2017-02-17 15:46:18,大石,市桥
3,NaN,NaN,ort1Ds99YgHuQpMCt-qgvggbYSmY,1201702041345056611,200,1,110,212.0,5,2017-02-04 13:45:05,2017-02-04 13:46:53,农讲所,公园前
4,NaN,NaN,ort1Ds23IHgU9gpe8NbszEOJqK4o,1201702112315089373,200,5,305,513.0,5,2017-02-11 23:15:08,2017-02-11 23:17:02,广州塔,珠江新城


In [3]:
df_suc = df[df['order_status'] == 5]
df_suc['time'] = pd.to_datetime(df_suc['entry_date'], format='%Y-%m-%d %H:%M:%S')
df_suc['hour'] = df_suc['time'].apply(lambda x: x.hour)

In [4]:
df[df.entry_date.isnull()].shape

(133823, 13)

In [5]:
df_suc.shape
df_suc.head()

(6371526, 15)

,user_id,account_id,owner_id,order_no,ticket_price,ticket_num,entry_station_code,exit_station_code,order_status,reg_date,entry_date,entry_station,exit_station,time,hour
0,NaN,NaN,ort1DswjyPc6pUEzGuJcgNepO9E0,1201702070627252238,800,3,317,3002.0,5,2017-02-07 06:27:25,2017-02-07 06:28:15,岗顶,机场南,2017-02-07 06:28:15,6
1,NaN,NaN,ort1Ds1jeqEkh9eHBc5FC0Pec_qI,1201702081959049604,200,1,107,0.0,5,2017-02-08 19:59:04,2017-02-08 20:01:15,陈家祠,NaN,2017-02-08 20:01:15,20
2,NaN,NaN,ort1Ds9TrCcK3YmzkCq3XoNqkIVA,1201702171545377297,400,1,310,312.0,5,2017-02-17 15:45:36,2017-02-17 15:46:18,大石,市桥,2017-02-17 15:46:18,15
3,NaN,NaN,ort1Ds99YgHuQpMCt-qgvggbYSmY,1201702041345056611,200,1,110,212.0,5,2017-02-04 13:45:05,2017-02-04 13:46:53,农讲所,公园前,2017-02-04 13:46:53,13
4,NaN,NaN,ort1Ds23IHgU9gpe8NbszEOJqK4o,1201702112315089373,200,5,305,513.0,5,2017-02-11 23:15:08,2017-02-11 23:17:02,广州塔,珠江新城,2017-02-11 23:17:02,23


In [21]:
tmp = df_suc.groupby(['entry_station', 'hour']).ticket_num.sum().reset_index()
starts = df_suc.groupby('entry_station').ticket_num.sum().sort_values(ascending=False)

In [29]:
tmp[tmp.entry_station == starts.index[0]][tmp.ticket_num > (tmp[tmp.entry_station == starts.index[0]].ticket_num.mean())*1.3]

,entry_station,hour,ticket_num
1036,广州南站,11,22903
1037,广州南站,12,23598
1038,广州南站,13,23288
1039,广州南站,14,25268
1040,广州南站,15,26769
1041,广州南站,16,26200
1042,广州南站,17,26755
1043,广州南站,18,24927
